# 🧠 Advanced LlamaIndex Workflow — Multi-Retriever + Streaming
This notebook demonstrates how to build a **multi-retriever RAG workflow** using the new `Workflows` API in LlamaIndex.

✅ Features:
- Multiple index ingestion
- Parallel retrieval
- Streaming LLM output

In [1]:

# 🧠 Install dependencies (run once)
!pip install llama-index-core llama-index-llms-openai llama-index-embeddings-huggingface


In [2]:
!pip install llama-index-llms-ollama llama-index-embeddings-cohere

In [3]:
import os
from dotenv import load_dotenv
from getpass import getpass

import nest_asyncio

nest_asyncio.apply()
load_dotenv()

True

In [4]:
CO_API_KEY = os.environ['CO_API_KEY'] or getpass("Enter your Cohere API key: ")

In [5]:
OPENAI_API_KEY = os.environ['OPENAI_API_KEY'] or getpass("Enter your OpenAI API key: ")

In [6]:
QDRANT_URL = os.environ['QDRANT_URL'] or getpass("Enter your Qdrant URL:")

In [7]:
QDRANT_API_KEY = os.environ['QDRANT_API_KEY'] or  getpass("Enter your Qdrant API Key:")

In [8]:
from llama_index.core.settings import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.cohere import CohereEmbedding

Settings.llm = Ollama(
    model="qwen2.5:7b",
    request_timeout=120.0,
    context_window=8000,
)

Settings.embed_model = CohereEmbedding(
    api_key=CO_API_KEY,
    model_name="embed-english-v3.0"
)

In [9]:

# 🧩 Imports
import asyncio
from llama_index.core.workflow import (
    Workflow, step, StartEvent, StopEvent, Context, Event,
)
from llama_index.llms.openai import OpenAI
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding


In [ ]:

# 🧩 Custom Event for multi-retrieval results
class MultiRetrieveEvent(Event):
    '''Event carrying retrieval results from multiple indexes.'''
    results: dict  # {index_name: [retrieved nodes]}


In [ ]:

# 🧠 Multi-Retriever Workflow Definition
class MultiRAGWorkflow(Workflow):
    '''Retrieval-Augmented Generation workflow with multiple retrievers and streaming output.'''

    @step
    async def ingest(self, ctx: Context, ev: StartEvent) -> StopEvent:
        '''Load documents from multiple directories and build vector indexes.'''
        dirs = ev.get("dirs") or []
        embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

        indexes = {}
        for d in dirs:
            print(f"📂 Building index for: {d}")
            docs = SimpleDirectoryReader(d).load_data()
            indexes[d] = VectorStoreIndex.from_documents(docs, embed_model=embed_model)

        await ctx.set("indexes", indexes)
        return StopEvent(result=f"✅ Built {len(indexes)} indexes.")

    @step
    async def retrieve(self, ctx: Context, ev: StartEvent) -> MultiRetrieveEvent:
        '''Retrieve from all indexes for the given query.'''
        query = ev.get("query")
        indexes = await ctx.get("indexes")
        if not query or not indexes:
            return MultiRetrieveEvent(results={})

        results = {}
        print(f"🔍 Retrieving for query: '{query}'\n")
        for name, idx in indexes.items():
            retriever = idx.as_retriever(similarity_top_k=2)
            nodes = await retriever.aretrieve(query)
            results[name] = nodes
            print(f"   → {name}: {len(nodes)} nodes retrieved")

        return MultiRetrieveEvent(results=results)

    @step
    async def generate_answer(self, ctx: Context, ev: MultiRetrieveEvent) -> StopEvent:
        '''Combine retrieved nodes and stream the LLM response.'''
        llm = OpenAI(model="gpt-4o", temperature=0.3, streaming=True)

        combined_context = ""
        for name, nodes in ev.results.items():
            combined_context += f"\n# Source: {name}\n"
            combined_context += "\n".join([n.text for n in nodes])

        prompt = f"""
        You are an expert assistant.
        Based on the following multi-source context, answer the user's question clearly.
        Context:
        {combined_context}
        """

        print("\n🧠 Generating answer (streaming tokens):\n")
        async for token in llm.astream_complete(prompt):
            print(token.delta, end="", flush=True)

        print("\n\n✅ Stream complete.")
        return StopEvent(result="Streaming completed.")


In [ ]:

# 🚀 Run the Multi-Retriever Workflow
async def main():
    w = MultiRAGWorkflow(timeout=60, verbose=True)
    await w.run(dirs=["./data/news", "./data/research"])  # Step 1: Ingest
    await w.run(query="Summarize the key findings on AI research trends.")  # Step 2: Query

await main()
